In [6]:
import pandas as pd
import numpy as np
import pysam

chrom = "chr21"
vcf_path="/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_regenie/vcf_qc_check/subset.vcf.gz"
vcf_file = pysam.VariantFile(vcf_path)
samples = list(vcf_file.header.samples)
print(f"Number of samples: {len(samples)}")

Number of samples: 2953


In [7]:
import pandas as pd
import numpy as np
import pysam
from concurrent.futures import ProcessPoolExecutor

# read sample information from the metadata file
cteph_jhrp4 = pd.read_csv("/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_agp3k/info/cteph_jhrp4_info.csv", sep=",")
cteph_30x = set(cteph_jhrp4[cteph_jhrp4["target_depth"] == "30x"]["sample_code"].values)
cteph_15x = set(cteph_jhrp4[cteph_jhrp4["target_depth"] == "15x"]["sample_code"].values)

In [8]:
def process_vcf(vcf_path, dp_thresholds, sample_codes):
    """
    Processes a VCF file to count the number of records that meet specified 
    depth of coverage (DP) thresholds for given sample codes.

    Args:
        vcf_path (str): Path to the input VCF file.
        dp_thresholds (list of int): A list of DP threshold values to evaluate.
        sample_codes (list of str): A list of sample codes to consider when 
            evaluating DP values.

    Returns:
        dict: A dictionary where keys are DP thresholds and values are the 
        counts of records that meet or exceed the corresponding DP threshold 
        for at least one of the specified sample codes.
    """
    vcf_in = pysam.VariantFile(vcf_path, 'r')  # open the VCF file for reading (r)
    counts = {dp: 0 for dp in dp_thresholds}  # initialize a dictionary to store the counts for each DP threshold

    for record in vcf_in.fetch():
        for dp in dp_thresholds:
            valid_sample = False
            for sample in record.samples.keys():
                if sample in sample_codes:
                    dp_value = record.samples[sample].get('DP', None)
                    if dp_value is not None and dp_value >= dp:
                        valid_sample = True
                        break
            if valid_sample:
                counts[dp] += 1

    vcf_in.close()
    return counts

In [9]:
def run_parallel_processing(vcf_path, dp_thresholds, sample_sets):
    """
    Executes parallel processing of VCF files for multiple sample sets using a process pool.

    Args:
        vcf_path (str): The file path to the VCF file to be processed.
        dp_thresholds (list): A list of depth thresholds to be applied during processing.
        sample_sets (list): A list of sample sets, where each sample set is processed independently.

    Returns:
        list: A list of results from processing each sample set. Each result corresponds to the output
        of the `process_vcf` function for a given sample set.
    """
    with ProcessPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(process_vcf, vcf_path, dp_thresholds, sample_set) for sample_set in sample_sets]
        results = [future.result() for future in futures]
    return results

In [10]:
# Define the DP thresholds to evaluate and the path to the VCF file
dp_thresholds = list(range(1, 31))
# Run parallel processing
results = run_parallel_processing(vcf_path, dp_thresholds, [cteph_30x, cteph_15x])

# Display the results
print("Results for 30x samples:", results[0])
print("Results for 15x samples:", results[1])

# Save the results to a CSV file
results_df = pd.DataFrame(results, index=["30x", "15x"])


Results for 30x samples: {1: 31, 2: 31, 3: 31, 4: 31, 5: 31, 6: 31, 7: 31, 8: 31, 9: 31, 10: 31, 11: 31, 12: 31, 13: 31, 14: 31, 15: 31, 16: 31, 17: 31, 18: 31, 19: 31, 20: 31, 21: 31, 22: 31, 23: 31, 24: 31, 25: 31, 26: 31, 27: 31, 28: 31, 29: 31, 30: 31}
Results for 15x samples: {1: 31, 2: 31, 3: 31, 4: 31, 5: 31, 6: 31, 7: 31, 8: 31, 9: 31, 10: 31, 11: 31, 12: 31, 13: 31, 14: 29, 15: 29, 16: 29, 17: 28, 18: 28, 19: 26, 20: 25, 21: 20, 22: 18, 23: 12, 24: 11, 25: 8, 26: 6, 27: 4, 28: 2, 29: 2, 30: 2}
